In [2]:
# from langchain_openai import AzureChatOpenAI
# from langchain.prompts import ChatPromptTemplate
# import os

# import truststore
# truststore.inject_into_ssl()
# import numpy as np
# import pandas as pd
# from tqdm import tqdm
# import time
# import seaborn as sns
# import numpy as np
# from sklearn.metrics import roc_curve, auc
# import matplotlib.pyplot as plt
# os.environ["APP_CLIENT_ID"] = "zizhang-chen-research-app"
# os.environ["APP_CLIENT_SECRET"] = "cQPcueFP7tDrimbf8NW2GAHcHeQa"
# from llm_idam_token_generator.idam_token_generator import get_idam_token
# from langchain_community.callbacks import get_openai_callback
# from langchain_openai import OpenAI
# import tiktoken
# # OpenAI Endpoint details
# OPENAI_ENDPOINT = "https://openai-llm-frontdoor-hma7evbthrd4cugn.a01.azurefd.net"
# # OPENAI_DEPLOYMENT_MODEL = "gpt-4-32k-beta"
# OPENAI_DEPLOYMENT_MODEL = "gpt-35-turbo-16k"
# OPENAI_AZURE_API_VERSION = "2023-12-01-preview"
# # OPENAI_AZURE_API_VERSION = '2024-02-01'
# OPENAI_TYPE="azure"

from langchain_openai import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
import os

# import truststore
# truststore.inject_into_ssl()
import numpy as np
import pandas as pd
from tqdm import tqdm
import time
import seaborn as sns
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
os.environ["APP_CLIENT_ID"] = "zizhang-chen-research-app"
os.environ["APP_CLIENT_SECRET"] = "cQPcueFP7tDrimbf8NW2GAHcHeQa"
# from llm_idam_token_generator.idam_token_generator import get_idam_token
from langchain_community.callbacks import get_openai_callback
from langchain_openai import OpenAI
# import tiktoken
# # OpenAI Endpoint details
# OPENAI_ENDPOINT = "https://openai-llm-frontdoor-hma7evbthrd4cugn.a01.azurefd.net"
# # OPENAI_DEPLOYMENT_MODEL = "gpt-4-32k-beta"
# OPENAI_DEPLOYMENT_MODEL = "gpt-35-turbo-16k"
# OPENAI_AZURE_API_VERSION = "2023-12-01-preview"
# # OPENAI_AZURE_API_VERSION = '2024-02-01'
# OPENAI_TYPE="azure"

In [3]:
# # help(AzureChatOpenAI)

# llm = AzureChatOpenAI(
#     api_key="xxx",  # This is not playing any role, but required as per OpenAI sdk. So any random could be passed.
#     azure_endpoint=OPENAI_ENDPOINT,
#     deployment_name=OPENAI_DEPLOYMENT_MODEL,
#     openai_api_version=OPENAI_AZURE_API_VERSION,
#     n = 2,
#     default_headers={
#         'Authorization': f'Bearer {get_idam_token()}',
#         'Content-Type': 'application/json'
#     }
# )

In [4]:
def generate_prompt_task(path):
    with open(path) as f:
        prompt = f.readlines()
    return prompt

def split_string_into_two_parts(text):
    lines = text.split('\n')
    first_part = lines[:2]
    second_part = lines[2:]

    first_part_string = '\n'.join(first_part)
    second_part_string = '\n'.join(second_part)
    
    return first_part_string, second_part_string


def generate_cot():

    cot = ["1. **Review Patient Profile:** Analyze age, sex, and medical event descriptions.",
           "2. **Analyze Results from Other lab test results:**"
           "3. **Evaluate Current Symptoms:** Consider how severe the symptoms are and whether they suggest worsening of condition or stability.",
           "4. **Assess Vital Signs:** Identify any vital signs that are outside normal ranges and what they indicate about the patient's current health status.",
           "5. **Interpret Laboratory Results:** Look for abnormalities in lab results that could signal a need for intensive care, such as organ dysfunction or severe infection."]
    
    return '\n'.join(cot)

def combin_prompt_cot(general_prompt, person_info, medical_event, multi):
    template_by_task = '**Task:**\n' + general_prompt + '\n'

    template_info = '**Patient age and demographic information:**\n' + person_info + '\n'

    template_events = '**Medical Events:**\n' + medical_event + '\n'
    
    template_events_answer_style = "**Answer requirement:**\nPlease only answer with 'Yes' or 'No'"

    multitask = multi

    return template_by_task + '\n' + '{events}' , template_info + '\n' + multi + template_events + '\n' + template_events_answer_style

def calculate_row_entropy(df):
    def entropy(row):
        counts = row.value_counts(normalize=True)
        return -np.sum(counts * np.log2(counts + np.finfo(float).eps)) 

    return df.apply(entropy, axis=1)

def num_tokens_from_string(string, encoding_model = 'gpt-3.5-turbo') -> int:
    encoding = tiktoken.encoding_for_model(encoding_model)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def generate_prompt_task(lab_test, lab_test_prediction):

    task = f"You are an experienced doctor. Based on the provided patient age, demographic information, medical events, and other relevant lab test results at the same time (make use of the other relevant lab test results wisely). Use your medical knowledge and reasoning to: {lab_test_prediction} right after the given medical events.\nPlease only answer with 'Yes' or 'No'. Yes if the patient lab results {lab_test} come back as normal, No if the patient lab results remain moderate or high. Please respond 'Yes' if it is at all plausible, only use 'No' if absolutely certain otherwise."
    return task

In [4]:
df = pd.read_csv('data/labtest_patient_description.csv')
description_list = df['description'].values
lab_test_list = [i.split('_')[1] for i in df.columns[2:7]]
lab_test_list
lab_test_prediction = [
    'Predict whether an anemia lab comes back as normal (>=120 g/L), mild (>=110 and <120 g/L), moderate (>=70 and <110 g/L), or severe (<70 g/L),',
    'Predict whether a hypoglycemia lab comes back as normal (>=3.9 mmol/L), mild (>=3.5 and <3.9 mmol/L), moderate (>=3 and <3.5 mmol/L), or severe (<3 mmol/L),',
    'Predict whether a hyponatremia lab comes back as normal (>=135 mmol/L), mild (>=130 and <135 mmol/L), moderate (>=125 and <130 mmol/L), or severe (<125 mmol/L),',
    'Predict whether a thrombocytopenia lab comes back as normal (>=150 10^9/L), mild (>=100 and <150 10^9/L), moderate (>=50 and <100 10^9/L), or severe (<50 10^9/L),',
    'Predict whether a hyperkalemia lab comes back as normal (<=5.5 mmol/L), mild (>5.5 and <=6mmol/L), moderate (>6 and <=7 mmol/L), or severe (>7 mmol/L),'
]

In [5]:
df

,patient_id,prediction_time,value_anemia,value_hypoglycemia,value_hyponatremia,value_thrombocytopenia,value_hyperkalemia,description
0,115969055,2014-12-03 05:36:00,1,0,1,0,1,The patient was 44 years old at the prediction...
1,115973689,2022-11-22 12:46:00,1,0,0,0,1,The patient was 61 years old at the prediction...
2,115968873,2008-03-12 03:29:00,1,1,1,0,0,The patient was 64 years old at the prediction...
3,115969135,2019-01-14 15:40:00,1,0,1,0,1,The patient was 65 years old at the prediction...
4,115972455,2014-06-22 06:17:00,0,1,0,0,0,The patient was 42 years old at the prediction...
...,...,...,...,...,...,...,...,...
95,115967430,2009-03-25 06:34:00,1,0,0,0,1,The patient was 41 years old at the prediction...
96,115968452,2020-12-12 05:30:00,1,1,0,0,0,The patient was 68 years old at the prediction...
97,115972907,2015-11-29 07:35:00,1,0,0,1,1,The patient was 59 years old at the prediction...
98,115972810,2020-01-14 04:18:00,0,0,1,1,1,The patient was 32 years old at the prediction...


In [6]:
df.columns

Index(['patient_id', 'prediction_time', 'value_anemia', 'value_hypoglycemia',
       'value_hyponatremia', 'value_thrombocytopenia', 'value_hyperkalemia',
       'description'],
      dtype='object')

In [7]:
def describe_conditions(row, current_idx):
    descriptions = {
        'value_anemia': 'moderate or severe (<120 g/L)' if row['value_anemia'] == 1 else 'normal (>=120 g/L)',
        'value_hypoglycemia': 'moderate or severe (<3.9 mmol/L)' if row['value_hypoglycemia'] == 1 else 'normal (>=3.9 mmol/L)',
        'value_hyponatremia': 'moderate or severe (<135 mmol/L)' if row['value_hyponatremia'] == 1 else 'normal (>=135 mmol/L),',
        'value_thrombocytopenia': 'moderate or severe (<150 10^9/L)' if row['value_thrombocytopenia'] == 1 else 'normal (>=150 10^9/L)',
        'value_hyperkalemia': 'moderate or severe (>5.5 mmol/L)' if row['value_hyperkalemia'] == 1 else 'normal ,(<=5.5 mmol/L)'
    }
    descriptions_keys = list(descriptions.keys())
    lab_test_list = ['anemia', 'hypoglycemia', 'hyponatremia', 'thrombocytopenia', 'hyperkalemia']
    descriptions_final = '**Relevant lab test results:**\n'
    for i in range(5):
        if i != current_idx:
            descriptions_final += lab_test_list[i] + ':\n' + descriptions[descriptions_keys[i]] + '\n\n'

    return descriptions_final

In [8]:
# for idx in [0]:
# for idx in range(5):
for idx in [3, 4]:
    out_csv_names = lab_test_list[idx].split('/')[-1].replace('.txt', '.csv')
    general_prompt = generate_prompt_task(lab_test=lab_test_list[idx], lab_test_prediction=lab_test_prediction[idx])
    gt_list = df['value_'+lab_test_list[idx]].values

    with get_openai_callback() as cb:

        answer_list_all = []

        for i in tqdm(range(len(df))):
        # for i in tqdm(range(50, 100)):
        # for i in tqdm(range(1)):
            row = df.loc[i, :]
            multi = describe_conditions(row, idx)
            answer_list_5 = []
            for _ in range(5):
            # for _ in range(1):
                current_description = description_list[i]
                person_info, medical_events = split_string_into_two_parts(current_description)
                c, e = combin_prompt_cot(general_prompt, person_info, medical_events, multi)
                
                prompt = ChatPromptTemplate.from_template(c)
                chain = prompt | llm
                result = chain.invoke({'events': e})
                answer_list_5.append(result.content)
            answer_list_all.append(answer_list_5)

        df_pred = pd.DataFrame(answer_list_all)
        df_pred.columns = ['pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5']
        df_pred['gt'] = list(gt_list)
        # df_pred['gt'] = list(gt_list[50:100])
        df_pred.to_csv(os.path.join('results/gpt35/cross_task_labtest/', out_csv_names + '.csv'))

  0%|          | 0/100 [00:00<?, ?it/s]PID:90681 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Request: POST https://openai-llm-frontdoor-hma7evbthrd4cugn.a01.azurefd.net/openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
PID:90681 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Request: POST https://openai-llm-frontdoor-hma7evbthrd4cugn.a01.azurefd.net/openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
PID:90681 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Request: POST https://openai-llm-frontdoor-hma7evbthrd4cugn.a01.azurefd.net/openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-12-01-preview "HTTP/1.1 200 OK"
PID:90681 INFO httpx - Client ID: zizhang-chen-research-app - HTTP Request: POST https://openai-llm-frontdoor-hma7evbthrd4cugn.a01.azurefd.net/openai/deployments/gpt-35-turbo-16k/chat/completions?api-version=2023-12-01-preview

In [9]:
len(answer_list_all)
df_pred = pd.DataFrame(answer_list_all)
df_pred.columns = ['pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5']
df_pred['gt'] = list(gt_list[0:50])
df_pred.to_csv(os.path.join('results/gpt4/cross_task_labtest/', out_csv_names + '_1.csv'))

ValueError: Length of values (50) does not match length of index (100)

In [ ]:
df1 = pd.read_csv('results/gpt4/cross_task_labtest/thrombocytopenia_1.csv')
df2 = pd.read_csv('results/gpt4/cross_task_labtest/thrombocytopenia.csv')
df3 = pd.concat([df1, df2])
df3 = df3.drop(['Unnamed: 0'], axis = 1)
df3 = df3.reset_index(drop=True)
df3.to_csv('results/gpt4/cross_task_labtest/thrombocytopenia.csv')